# Generate circular tree for Fig. S5

* Uses previous DESeq2 analysis and phylogenetic tree to plot differential abundance


In [ ]:
out_Dir = '~/Hyphosphere/data/3Exp/figures'

work_dir = '~/Hyphosphere/data/3Exp/DESeq'

# phyloseq - full object with tree
physeq_file = '~/Hyphosphere/data/3Exp/phyloseq/3Exp_phyloseq_thresh_wtree_experimental_woContam.rds'

In [ ]:
library(RColorBrewer)
library(plyr)
library(tidyr)
library(dplyr)
library(reshape2)
library(phyloseq)
library(ggplot2)
library(ggtree)
library(gridExtra)

In [ ]:
g_legend <- function(a.gplot){
  tmp <- ggplot_gtable(ggplot_build(a.gplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)
}



In [ ]:
ps = readRDS(physeq_file) %>% subset_taxa(., !is.na(Phylum)) %>%
    # remove non-experimental controls
    
    subset_samples(., SampleType != "GV" & Experiment %in% c(1,2) & Fungus == "Gv" & SampleType %in% c("CS","CH") & Soil != "Sand") %>%

     # re-normalize after subsetting taxa
    transform_sample_counts(., function(x) x/sum(x))

ps


In [ ]:
ps.m = ps %>%
    psmelt() %>%
    group_by(OTU, Soil, Experiment) %>%
    summarise(mAbund = mean(Abundance)) %>%
    ungroup() %>% 
    group_by(OTU) %>%
    summarise(smAbund = sum(mAbund)) 
head(ps.m)

### Top 200 most abundant taxa

In [ ]:
topTax = ps.m %>% 
    arrange(desc(smAbund)) %>%
    .[1:200,] %>%
    .$OTU %>%
    as.character()

topTax
            

In [ ]:
rm(ps.m)

In [ ]:
# Filter to the top OTUs identified above

physeq.thresh <- prune_taxa(topTax, ps)

physeq.thresh


In [ ]:

MyTree =  physeq.thresh %>% phy_tree


In [ ]:
TreeTax = taxa_names(physeq.thresh)

In [ ]:
df.tax = physeq.thresh %>% tax_table %>% as.data.frame
df.tax$OTU = df.tax %>% row.names

df.tax = mutate(df.tax, TaxLabel = paste(Family, Genus, sep = "_")) %>%
    select(OTU, TaxLabel, Phylum, Class, Order, Family, Genus)




## Relative Abundance data

In [ ]:
mdf = psmelt(physeq.thresh)
mdf.byOTU = ddply(mdf, .(Soil, OTU, Experiment), summarize, mean_Abundance = mean(Abundance)) 
head(mdf.byOTU)

### log2FoldChange data

In [ ]:
readfile = file.path(work_dir, 'dfl2fc-Exp1.txt')
df.l2fc.Exp1 = read.table(readfile, header = TRUE, sep= '\t')

readfile = file.path(work_dir, 'dfl2fc-Exp2.txt')
df.l2fc.Exp2 = read.table(readfile, header = TRUE, sep= '\t')

df.l2fc = bind_rows(df.l2fc.Exp1, df.l2fc.Exp2)
df.l2fc = df.l2fc %>% separate(Subset, c("Soil","Fungus")) %>% filter(Contrast %in% c("CH_CS", "CH_BS", "CH_BP", "RH_RS") & Fungus == "Gv")


In [ ]:
head(df.l2fc)

## Get color scheme for tree

In [ ]:
tmp = left_join(df.l2fc, mdf.byOTU) %>%
    select(OTU, Soil, Experiment, padj, Contrast, mean_Abundance, log2FoldChange, Phylum, Order, Class, Family, Genus) %>%
    filter(OTU %in% TreeTax)


tmp$Phylum = factor(tmp$Phylum)
tmp$Order = factor(tmp$Order)
tmp$Class = factor(tmp$Class)

#Link phylum colors 
FigPhyla = tmp$Phylum %>% levels() %>% as.character()
FigClass = tmp$Class %>% levels() %>% as.character()
FigOrder = tmp$Order %>% levels() %>% as.character()
FigOrder

ColorCount = length(FigOrder)
getPalette = colorRampPalette(brewer.pal(8, "Dark2"))
Ocs = getPalette(ColorCount)
Ocs


ColorCount = length(FigPhyla)
getPalette = colorRampPalette(brewer.pal(8, "Dark2"))
Pcs = getPalette(ColorCount)
Pcs

ColorCount = length(FigClass)
getPalette = colorRampPalette(brewer.pal(8, "Dark2"))
Ccs = getPalette(ColorCount)
Ccs


# Pcs = brewer.pal(12,"Set3")

#Set lookup to 
Olookup = cbind(FigOrder, Ocs) %>% as.data.frame()
Plookup = cbind(FigPhyla, Pcs) %>% as.data.frame()
Clookup = cbind(FigClass, Ccs) %>% as.data.frame()

tmp = left_join(tmp, Plookup, by = c("Phylum" = "FigPhyla"))
tmp = left_join(tmp, Olookup, by = c("Order" = "FigOrder")) 
tmp = left_join(tmp, Clookup, by = c("Class" = "FigClass")) 



tmp %>% head()

# Test basic ggtree

## add genus labels

In [ ]:
df.tax = df.tax %>%
    mutate(GenusLabel = ifelse(!is.na(Genus), paste(Genus), 
                        ifelse(!is.na(Family), paste('Uncl. ', Family, sep = ""), 
                        ifelse(!is.na(Order), paste('Uncl. ', Order, sep = ""),
                        ifelse(!is.na(Class), paste('Uncl. ', Class, sep = ""), paste("Uncl. ", Phylum, sep = "")))))) %>%
    mutate(GenusLabel = ifelse(GenusLabel == 'Allorhizobium-Neorhizobium-Pararhizobium-Rhizobium', 'Allo-Neo-Para-Rhizobium', GenusLabel))
head(df.tax)

# Try circular tree

In [ ]:
mdf.byOTU %>% filter(OTU == 'ASV5')

### Simple two color heat map

In [ ]:
head(df.tax)

### For multicolor heatmap

In [ ]:
df.circ = df.l2fc %>%
    filter(Fungus == "Gv") %>%
    mutate(sig = ifelse(padj >= 0.05 | is.na(padj), "ns", 
                 ifelse(log2FoldChange <= 0, "ns", "sig")), 
           variable = paste(Soil,Experiment, sep = '')) %>%

    mutate(sig = ifelse(Contrast == "CH_CS" & sig == "sig", 1,
                       ifelse(Contrast == "CH_BS" & sig == "sig", 2,
                           ifelse(Contrast == "CH_BP" & sig == "sig", 3,  
                                 ifelse(Contrast == "RH_RS" & sig == "sig", 4,
                                       "ns"))))) %>%
    select(OTU, variable, Contrast, sig) %>%
    dcast(OTU ~ variable + Contrast) 
head(df.circ)
df.circ = df.circ %>%
    mutate(Blank1 = "blank", 
          Blank2 = "blank")

rownames(df.circ) = df.circ$OTU
df.circ = df.circ %>% select(-OTU)
#head(df.circ)
head(df.circ)

In [ ]:
testcirc = df.circ %>% select("Dryden2_CH_CS","Dryden2_CH_BS", "Dryden2_CH_BP","Dryden2_RH_RS","Blank1", 
                              "Florence2_CH_CS","Florence2_CH_BS", "Florence2_CH_BP","Florence2_RH_RS", "Blank2",
                             "Pendelton2_CH_CS","Pendelton2_CH_BS", "Pendelton2_CH_BP","Pendelton2_RH_RS")
head(testcirc)

In [ ]:
testcirc %>% filter(Dryden2_RH_RS == 4)

In [ ]:
ColorCount = df.tax$Phylum %>% unique() %>% length()
getPalette = colorRampPalette(brewer.pal(8, "Dark2"))
Pcs = getPalette(ColorCount)

cs = (brewer.pal(4, 'PuOr'))

In [ ]:
p  = ggtree(MyTree, layout = "circular", branch.length = "none") %<+% df.tax
#p = ggtree(MyTree)%<+% df.tax
p = p +  
geom_tippoint(aes(color=Phylum), size=1) +
geom_tiplab(aes(label = GenusLabel),hjust = -.1, align = TRUE, size = 1)+

    scale_color_manual(values = Pcs, guide = guide_legend(ncol = 3))

p = gheatmap(p, testcirc, offset=10, width=0.4, colnames_position = NULL, 
         colnames_angle=90, hjust=0, colnames_offset_y = .25, font.size = 2) +
       scale_fill_manual(breaks=c( "1", "2","3","4", "blank", "ns" ),values=c(cs[1],cs[2],cs[3],cs[4], "White", "Light Grey")) +
    theme(legend.position = 'bottom')

#            # scale_fill_viridis_d(option="D", name='discrete\nvalue')
#             # scale_color_manual(breaks=c("ns", "sig"), values=c("White", "Red"), name="HASV")


# #p2 <- p1 + new_scale_fill()
p

outfile = file.path(out_Dir, 'CircularTree_controls.pdf')
pdf(outfile, width = 7.036, height = 9, useDingbats = FALSE)
p
dev.off()